## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2022.

# ECBM E4040 - Assignment 2- Task 5: Kaggle Open-ended Competition

Kaggle is a platform for predictive modelling and analytics competitions in which companies and researchers post data and statisticians and data miners compete to produce the best models for predicting and describing the data.

If you don't have a Kaggle account, feel free to join at [www.kaggle.com](https://www.kaggle.com). To let the CAs do the grading more conveniently, please __use Lionmail to join Kaggle__ and __use UNI as your username__.

The competition is located here: https://www.kaggle.com/t/d908ef03b7244102a1e006516a6555a6

You can find detailed description about this in-class competition on the website above. Please read carefully and follow the instructions.

<span style="color:red">__TODO__:</span>

- Train a custom model for the bottle dataset classification problem. You are free to use any methods taught in the class or found by yourself on the Internet (ALWAYS provide reference to the source). General training methods include:
  - Dropout
  - Batch normalization
  - Early stopping
  - l1-norm & l2-norm penalization
- You are given the test set to generate your predictions (70% public + 30% private, but you don't know which ones are public/private). Students should achieve an accuracy on the public test set of at least 70%. Two points will be deducted for each 1% below 70% accuracy threshold (i.e. 65% accuracy will have 10 points deducted). The accuracy will be shown on the public leaderboard once you submit your prediction .csv file. The private leaderboard will be released after the competition. The final ranking is based on the private leaderboard result, not the public leaderboard.


NOTE: 
* Report your results on the Kaggle, for comparison with other students' optimal results (you can do this several times). 
* Save your best model.

__Hint__: You can start from what you implemented in task 4. Students are allowed to use pretrained networks, and utilize transfer learning. 

## Useful Information: 

- Unzip zip files in GCP or acquire administrator permission for other application installation: When you upload your dataset to your vm instances, you may want to unzip your files. However, unzip command is not built in. To use `sudo apt install unzip` or for future applications installation, you need to: 
  - Change username to default (or just restart the vm instance)
  - Type `sudo su` to get root
  - You can remove sudo for the following installation commands (e.g. `apt install unzip`).
- If you meet kernel crash (or the running never ends), you might consider using a larger memory CPU. Especially if you include large network structure like VGG, 15GB memory or more CPU is recommended
- Some python libraries that you might need to install first: pandas, scikit-learn. there are **2 OPTIONS** that you can use to install them:
  - In the envTF24 environment in linux interface, type: `pip install [package name]` 
  - In the jupyter notebook (i.e. this file), type `!pip install [package name]`. You’d better restart the virtual environment, even the instance to get these packages functional.
- You might need extra pip libraries to handle dataset, include network, etc. You can follow step 3 to install them.

## HW Submission Details:

There are two components to reporting the results of this task: 

**(A) Submission (up to 20 submissions each day) of the .csv prediction file through the Kaggle platform**. You should start doing this __VERY early__, so that students can compare their work as they are making progress with model optimization.

**(B) Submitting your best CNN model through Github Classroom repo.**

**Note** that assignments are submitted through github classroom only. All code for training your kaggle model should be done in this task 5 jupyter notebook, or in a user defined module (.py file) that is imported for use in the jupyter notebook.

<span style="color:red">__Submission content:__</span>

(i) In your Assignment 2 submission folder, create a subfolder called __KaggleModel__. Save your best model using `model.save()`. This will generate a `saved_model.pb` file, a folder called `variables`, and a folder called `checkpoints` all inside the __KaggleModel__ folder. Only upload your best model to GitHub classroom. 

(ii) <span style="color:red">If your saved model exceeds 100 MB, do not upload it to GitHub classroom (.gitignore it or you will get an error when pushing).</span> Upload it instead to Google Drive and explicitly provide the link under the 'Save your best model' cell. 

(iii) Remember to delete any intermediate results, we only want your best model. Do not upload any data files. The instructors will rerun the uploaded best model and verify against the score which you reported on the Kaggle.

**The top 10 final submissions of the Kaggle competition will receive up to 10 bonus points proportional to the private test accuracy.**

## Load Data

In [1]:
#Generate dataset
import os
import pandas as pd
import numpy as np
from PIL import Image
import warnings

#Load Training images and labels
train_directory = "C:\\Users\\highm\\Downloads\\e4040-2022fall-assign2-zy2523\\data\\kaggle_train_128\\train_128" #TODO: Enter path for train128 folder (hint: use os.getcwd())
image_list=[]
label_list=[]
for sub_dir in os.listdir(train_directory):
    print("Reading folder {}".format(sub_dir))
    sub_dir_name=os.path.join(train_directory,sub_dir)
    for file in os.listdir(sub_dir_name):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_list.append(np.array(Image.open(os.path.join(sub_dir_name,file))))
            label_list.append(int(sub_dir))
X_train=np.array(image_list)
y_train=np.array(label_list)
warnings.simplefilter('ignore')
#Load Test images
test_directory = "C:\\Users\\highm\\Downloads\\e4040-2022fall-assign2-zy2523\\data\\kaggle_test_128\\test_128"#TODO: Enter path for test128 folder (hint: use os.getcwd())
test_image_list=[]
test_df = pd.DataFrame([], columns=['Id', 'X'])
print("Reading Test Images")
for file in os.listdir(test_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        test_df = test_df.append({
            'Id': filename,
            'X': np.array(Image.open(os.path.join(test_directory,file)))
        }, ignore_index=True)
        
test_df['s'] = [int(x.split('.')[0]) for x in test_df['Id']]
test_df = test_df.sort_values(by=['s'])
test_df = test_df.drop(columns=['s'])
X_test = np.stack(test_df['X'])


print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)

Reading folder 0
Reading folder 1
Reading folder 2
Reading folder 3
Reading folder 4
Reading Test Images
Training data shape:  (15000, 128, 128, 3)
Training labels shape:  (15000,)
Test data shape:  (3500, 128, 128, 3)


In [ ]:
#import zipfile
#import os
#files = zipfile.ZipFile("ecbm-e4040fall2022-assignment-2-task-5.zip",'r')
#files.extractall(os.getcwd())

## Build and Train Your Model Here

In [2]:
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
import datetime
from tensorflow.keras import regularizers

In [3]:
#convert the images into float32 type
X_train = np.array(X_train, dtype = np.float32)

#shuffle the samples
rand = np.arange(y_train.shape[0])
np.random.shuffle(rand)
X_train = X_train[rand]
y_train = y_train[rand]


In [4]:
x_train1 = X_train[:-1500]
y_train1 = y_train[:-1500]
x_val = X_train[-1500:]
y_val = y_train[-1500:]

print('x_train: ',x_train1.shape)
print('y_train: ',y_train1.shape)
print('x_val: ',x_val.shape)
print('y_val: ',y_val.shape)

x_train:  (13500, 128, 128, 3)
y_train:  (13500,)
x_val:  (1500, 128, 128, 3)
y_val:  (1500,)


In [5]:
# YOUR CODE HERE
def create_model():
    model = tf.keras.models.Sequential([
        
        # Conv_Layer 1
        tf.keras.layers.Conv2D(filters=5, kernel_size=7, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2)),
        
        # Conv_Layer 2
        tf.keras.layers.Conv2D(filters=7, kernel_size=5, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2)),
        
        # Fully Connected Layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.Dense(5, activation='softmax')
        
    ])
     
    return model

In [6]:
#Create the model, compile the model, and fit it
model_test = create_model()
model_test.compile(optimizer='Nadam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_test.fit(x=x_train1, 
               y=y_train1, 
               batch_size=256, 
               epochs=30, 
               validation_data=(x_val, y_val))

Epoch 1/30
53/53 [==============================] - 60s 1s/step - loss: 8.5287 - accuracy: 0.5924 - val_loss: 4.5120 - val_accuracy: 0.5967
Epoch 2/30
53/53 [==============================] - 60s 1s/step - loss: 2.8141 - accuracy: 0.7881 - val_loss: 2.2511 - val_accuracy: 0.7020
Epoch 3/30
53/53 [==============================] - 59s 1s/step - loss: 1.6446 - accuracy: 0.8319 - val_loss: 2.1621 - val_accuracy: 0.5760
Epoch 4/30
53/53 [==============================] - 58s 1s/step - loss: 1.2398 - accuracy: 0.8499 - val_loss: 1.2254 - val_accuracy: 0.8207
Epoch 5/30
53/53 [==============================] - 58s 1s/step - loss: 1.0163 - accuracy: 0.8702 - val_loss: 1.0795 - val_accuracy: 0.8300
Epoch 6/30
53/53 [==============================] - 58s 1s/step - loss: 0.8809 - accuracy: 0.8817 - val_loss: 1.0698 - val_accuracy: 0.8060
Epoch 7/30
53/53 [==============================] - 58s 1s/step - loss: 0.7989 - accuracy: 0.8930 - val_loss: 0.9388 - val_accuracy: 0.8267
Epoch 8/30
53/53 [==

## Save your best model

**Link to large model on Google Drive: [insert link here]** (if model exceeds 100 MB, else upload to GitHub classroom)

In [9]:
# YOUR CODE HERE
model_test.save('best_model')

INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


## Generate .csv file for Kaggle

The following code snippet can be an example used to generate your prediction .csv file.

NOTE: If your Kaggle results are indicating random performance, then it's likely that the indices of your csv predictions are misaligned.

In [7]:
prediction = model_test.predict(X_test)
ans = np.zeros([prediction.shape[0],])

for i,val in enumerate(prediction):
    max_num = np.max(val)
    ind = np.where(val == max_num)
    ans[i] = ind[0]

ans = np.array(ans, dtype=int)

110/110 [==============================] - 3s 30ms/step


In [8]:
import csv
with open('predicted.csv','w',newline='') as csvfile:
    fieldnames = ['Id','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index,l in enumerate(ans):
        filename = str(index) + '.png'
        label = str(l)
        writer.writerow({'Id': filename, 'label': label})